In [149]:
## Task 1

""" The message in this example is expressed as a list of integers. Each integer corresponds to a separate character 
in the original message. The mapping between characters and `decoded` integers is:

 - 0-9: numbers 0-9
 - 10-35: letters a-z (only lowercase is used)
 - 36: space
 
Example: "abc def" would be converted to [10, 11, 12, 36, 13, 14, 15].

Upon receiving the *encoded* message from other party, we need to decode it (character by character)
using your RSA private key, and then convert the resulting list of integers back to a string 
using the table, above.

First, we (the receiver) generate the key: (d,e,N) = (169,25,299) and announcd (e,N) part of it, publicly.
The sender encodes the message (first onto integers, following by the encryption part of RSA) and sends it
to us."""

message_received = [
    292, 290, 218, 55, 127, 174, 171, 127, 112, 24,
    251, 248, 127, 132, 218, 213, 24, 251, 248, 174, 55,
    53, 127, 233, 24, 268, 24, 251, 248
]

message_to_send = "green"

# Define the correspondance table between integers and strings/integers 
dict1 = {i:chr(i+48) for i in range(0,10)}
dict2 = {i+9:chr(i+96) for i in range(1,27)}
dict3 = {36:chr(32)}
dict = Merge(dict1,dict2,dict3)

# Define the correspondance table between strings/integers and integers (reverse of the table above)
dict1 = {chr(i+48):i for i in range(0,10)}
dict2 = {chr(i+96):i+9 for i in range(1,27)}
dict3 = {chr(32):36}
dict_rev = Merge(dict1,dict2,dict3)

# Merge to concatenate dictionaries
def Merge(dict_1, dict_2, dict_3):
    result = dict_1 | dict_2 | dict_3
    return result

### The main code ###

def decrypt(message, d, N, dict):
    """ Decrypt an encoded message. 
 
    Args:
        message (list[int]): A list of integers representing the secret message.
        NOTE: Each integer in the list represents a different character which needs to be
        converted to to a string using the table, shared by both parties.

        d (int): The (private) portion of the RSA key; d in (d,e,N)
        N (int): The modulus of the RSA key.
 
    Returns: The decoded message in string form. """
    
    decrypted_integers = [] # First decrypt the message (int to int)
    for c in message:
        decrypted_integers.append((c ** d) % N)
       
    decoded_message = [] # Then, decode the message in a string using the dictionary (int t str)
    for c in decrypted_integers:
        decoded_message.append(dict[c])
    
    return decoded_message


[d,e,N] = [169,25,299]    
decrypted_message = decrypt(message_received, d, N, dict)    
print('The message received is: '+''.join(decrypted_message)) # The message received in text
    
def encrypt(message, e, N):
    """ Encrypt a message 
    Args:
        message (str): A string representation of the message to send. It contains only the characters a-z 
        (lowercase), numbers 0-9, and spaces.
        e, N (int): The public portion of the RSA key (e, N) sent by the other party for us to encode.
        
 
    Returns: The message, encoded using the public key as a list of integers. """
    encoded_message = [] # First, encode the string message in a List of integers using the dictionary (str to int)
    for c in message:
        encoded_message.append(int(dict_rev[c]))
    
    encrypted_integers = [] # Then encrypt the message (int to int)
    for c in encoded_message:
        encrypted_integers.append((c ** e) % N)
       
    return encrypted_integers


[e,N] = [29,91] 
encrypted_message = encrypt(message_to_send, e, N) 
print('The message to be sent in original text is: '+ message_to_send) # The text message to be sent
print('The encrypted message to send is: '+ str(encrypted_message)) # The encrypted message to be sent in integers

The message received is: what is your favourite colour
The message to be sent in original text is: green
The encrypted message to send is: [74, 27, 14, 14, 4]


In [ ]:
# Challenge 1
import timeit
start = timeit.default_timer()  
from qiskit import IBMQ
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor

IBMQ.enable_account(XXXXXX) # Enter your API token here
provider = IBMQ.get_provider(hub='ibm-q')

backend = provider.get_backend('ibmq_qasm_simulator') # Specify the quantum device

print('\n Shors Algorithm')
print('--------------------')
print('\nExecuting...\n')

factors = Shor(QuantumInstance(backend, shots = 50)) 

result_dict = factors.factor(N = 91, a = 8) # Where N is the integer to be factored
result = result_dict.factors

print(result)
stop = timeit.default_timer()
print('Time: ', stop - start)
print('\nPress any key to close')
input()




 Shors Algorithm
--------------------

Executing...

[[7, 13]]
Time:  10663.000639525

Press any key to close


### Challenge 1: Shor's Algorithm requirements to run on a quantum computer
We saw before that to implement Shor's algorithm one need of $\mathcal{O}(logN)$ qubits for the number 
and the same for the ancillary qubits, initially prepared in $\vert 0 \rangle$. According to a paper by Haner et al. (arXiv:1611.07995) implementation of Shor's algorithm for factoring an N-bit integer can be done using $2N+2$ qubits. 
The circuit depth and the overall gate count are in $\mathcal{O}(N^{3})$ and $\mathcal{O}(N^{3} log(N))$, respectively. Thus, we see that for the integer $91$, 
for example, $N = 7$ meaning that we would need $16$ qubits machine and run on it, many times. 
